In [1]:
import pandas as pd
import numpy as np

# 1. Load
df = pd.read_excel("AmericanExpress_Data.xlsx")

# 2. Clean
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("($M)", "M", regex=False)
    .str.replace("%", "perc", regex=False)
)

# Check what columns actually are
print("Cleaned column names:", df.columns.tolist())

Cleaned column names: ['Period', 'Year', 'Quarter', 'Region', 'Segment', 'Revenue_M', 'Interest_Income_M', 'Expenses_M', 'Net_Income_M', 'Active_Cards_(M)', 'Avg_Spend_per_Card_($)', 'Delinquency_perc']


In [4]:
import pandas as pd
import numpy as np

# 1. Load
df = pd.read_excel("AmericanExpress_Data.xlsx")

# 2. Clean column names
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("($M)", "M", regex=False)
    .str.replace("%", "perc", regex=False)
    .str.replace("$", "", regex=False)  # remove $ sign safely
)

# 3. Explicit rename for consistency
df = df.rename(columns={
    "Active_Cards_(M)": "Active_Cards_M",
    "Avg_Spend_per_Card_(_)": "Avg_Spend_per_Card"  # optional cleanup
})

print("Final column names:", df.columns.tolist())

# 4. Ensure numeric columns
for col in ["Revenue_M", "Net_Income_M", "Active_Cards_M", "Delinquency_perc"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 5. Core KPI calculations
df["Net_Income_Margin_perc"] = (df["Net_Income_M"] / df["Revenue_M"]) * 100
df["YoY_Revenue_Growth_perc"] = (
    df.groupby(["Region", "Segment", "Quarter"])["Revenue_M"]
    .pct_change(periods=1) * 100
)
df["Revenue_per_ActiveCard"] = df["Revenue_M"].div(df["Active_Cards_M"]).replace([np.inf, -np.inf], np.nan)
df["Delinq_to_Revenue_perc"] = (df["Delinquency_perc"] / df["Revenue_M"]) * 100

# 6. Export
df.to_excel("Amex_Cleaned_Data2.xlsx", index=False)
print("Data cleaning & KPI calculations complete! File saved as Amex_Cleaned_Data2.xlsx")


✅ Final column names: ['Period', 'Year', 'Quarter', 'Region', 'Segment', 'Revenue_M', 'Interest_Income_M', 'Expenses_M', 'Net_Income_M', 'Active_Cards_M', 'Avg_Spend_per_Card_()', 'Delinquency_perc']
Data cleaning & KPI calculations complete! File saved as Amex_Cleaned_Data2.xlsx


In [7]:
print(df.dtypes)


Period                      object
Year                         int64
Quarter                     object
Region                      object
Segment                     object
Revenue_M                    int64
Interest_Income_M            int64
Expenses_M                   int64
Net_Income_M                 int64
Active_Cards_M               int64
Avg_Spend_per_Card_()        int64
Delinquency_perc           float64
Net_Income_Margin_perc     float64
YoY_Revenue_Growth_perc    float64
Revenue_per_ActiveCard     float64
Delinq_to_Revenue_perc     float64
dtype: object
